In [1]:
import numpy as np
import sys
import torch
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from time import time

In [2]:
torch.cuda.set_device(2)
print(torch.cuda.current_device())
print(torch.__version__)

2
1.0.0


In [3]:
# random hull
def rand_hull(n_points, dim, device="cuda", dtype=torch.float32):
    V = np.random.rand(n_points, dim)
    mesh = Delaunay(V)
    E = mesh.simplices
    E = torch.LongTensor(E).to(device)

    # normalize V, and create V as leaf tensor
    V_bb = np.max(V, axis=0) - np.min(V, axis=0)
    V_c = (np.max(V, axis=0) + np.min(V, axis=0)) / 2
    V -= V_c
    V /= 1.5*V_bb
    V += 0.5
    V = torch.tensor(V, device=device, dtype=dtype, requires_grad=True)
    return V, E

def plot_F(F):
    F = F.squeeze()
    if torch.sum(torch.isnan(F)) > 0:
        nnan = torch.sum(torch.isnan(F))
        F[torch.isnan(F)] = 0
        print("Padding {} nan terms to zero.".format(nnan))

    F_ = F.cpu().detach().numpy()
    f_ = torch.irfft(F, dim, signal_sizes=res).squeeze().cpu().detach().numpy()
    plt.figure(figsize=(6,6))
    plt.imshow(f_.T, origin='lower')

    plt.colorbar()
    plt.show()

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch.autograd.function import once_differentiable
from math import pi

from utils import fftfreqs, simplex_content, triangulate_interior, permute_seq, coalesce_update
from math import ceil, factorial

class SimplexFT(Function):
    """
    Fourier transform for signal defined on a j-simplex set in R^n space
    :param V: vertex tensor. float tensor of shape (n_vertex, n_dims)
    :param E: element tensor. int tensor of shape (n_elem, j or j+1)
              if j cols, triangulate/tetrahedronize interior first.
    :param D: int ndarray of shape (n_elem, n_channel)
    :param res: n_dims int tuple of number of frequency modes
    :param t: n_dims tuple of period in each dimension
    :param j: dimension of simplex set
    :param mode: normalization mode.
                 'density' for preserving density, 'mass' for preserving mass
    :return: F: ndarray of shape (res[0], res[1], ..., res[-1]/2, n_channel)
                last dimension is halfed since the signal is assumed to be real
    """
    @staticmethod
    def forward(ctx, V, E, D, res, t, j, elem_batch=100, mode='density'):
        ## boiler-plate
        ctx.mark_non_differentiable(E, D) # mark non-differentiable
        ctx.res = res
        ctx.t = t
        ctx.j = j
        ctx.mode = mode
        ctx.n_dims = V.shape[1]
        ctx.elem_batch = elem_batch
        # compute content array
        C = factorial(j) * simplex_content(V, E) # [n_elem, 1]
        ctx.save_for_backward(V, E, D, C)

        ## compute frequencies F
        n_dims = ctx.n_dims
        assert(n_dims == len(res))  # consistent spacial dimensionality
        assert(E.shape[0] == D.shape[0])  # consistent vertex numbers
        assert(mode in ['density', 'mass'])

        # number of columns in E
        subdim = E.shape[1] == j and n_dims == j
        assert (E.shape[1] == j+1 or subdim)
        if subdim:
            E = triangulate_interior(V, E)
        n_elem = E.shape[0]
        n_vert = V.shape[0]
        n_channel = D.shape[1]

        # frequency tensor
        omega = fftfreqs(res, dtype=V.dtype).to(V.device) # [dim0, dim1, dim2, d]

        # normalize frequencies
        for dim in range(n_dims):
            omega[..., dim] *= 2 * pi / t[dim]

        # initialize output F
        F_shape = list(omega.shape)[:-1]
        F_shape += [n_channel, 2]
        F = torch.zeros(*F_shape, dtype=V.dtype, device=V.device) # [dimX, dimY, dimZ, n_chan, 2] 2: real/imag

        # compute element-point tensor
        P = V[E] # [n_elem, j+1, d]

        # loop over element batches
        for idx in range(ceil(n_elem/elem_batch)):
            id_start = idx * elem_batch
            id_end = min((idx+1) * elem_batch, n_elem)
            Xi = P[id_start:id_end] # [elem_batch, j+1, d]
            Di = D[id_start:id_end] # [elem_batch, n_channel]
            Ci = C[id_start:id_end] # [elem_batch, 1]
            CDi = Ci * Di # [elem_batch, n_channel]
            sig = torch.einsum('bjd,...d->bj...', (Xi, omega)) 
            sig = torch.unsqueeze(sig, dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1]
            esig = torch.stack((torch.cos(sig), -torch.sin(sig)), dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 2]
            sig = torch.unsqueeze(sig, dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
            denom = torch.ones_like(sig) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
            for dim in range(1, j+1):
                seq = permute_seq(dim, j+1)
                denom *= sig - sig[:, seq]
            tmp = torch.sum(esig / denom, dim=1) # [elem_batch, dimX, dimY, dimZ, 1, 2]
            CDi.unsqueeze_(-1) # [elem_batch, n_channel, 1]
            for _ in range(n_dims): # unsqueeze to broadcast
                CDi.unsqueeze_(dim=1) # [elem_batch, 1, 1, 1, n_channel, 1]
            tmp *= CDi # [elem_batch, dimX, dimY, dimZ, n_channel, 2]
            Fi = torch.sum(tmp, dim=0, keepdim=False) # [dimX, dimY, dimZ, n_channel, 2]
            Fi[tuple([0] * n_dims)] = - 1 / factorial(j) * torch.sum(CDi, dim=0).unsqueeze(dim=-1)
            F += Fi
        # multiply by (i)**j
        if j == 0:
            pass
        elif j == 1:
            F = F[..., [1, 0]]
            F[..., 0] = -F[..., 0]
        elif j == 2:
            F *= -1
        elif j == 3:
            F = F[..., [1, 0]]
            F[..., 1] = -F[..., 1]

        if mode == 'density':
            if not np.array_equal(res, res[0]*np.ones(len(res))):
                print("WARNING: density preserving mode not correctly implemented if not all res are equal")
            F *= res[0] ** j
        return F

    @staticmethod
    @once_differentiable
    def backward(ctx, dF):
        """
        :param dF: per-frequency sensitivity from downstream layers of shape [dimX, dimY, dimZ, n_channel, 2]
        """
        if ctx.needs_input_grad[0]:
            V, E, D, C = ctx.saved_tensors

            n_dims = V.shape[1]
            assert(n_dims == 2) # backwards not implemented for other dims yet
            assert(ctx.j == 2) # not yet implemented for other simplices

            n_elem = E.shape[0]
            n_vert = V.shape[0]
            n_channel = D.shape[1]

            # recover context
            res = ctx.res
            t = ctx.t
            j = ctx.j
            mode = ctx.mode
            n_dims = ctx.n_dims
            elem_batch = ctx.elem_batch

            # frequency tensor
            omega = fftfreqs(res, dtype=V.dtype).to(V.device) # [dim0, dim1, dim2, d]

            # normalize frequencies
            for dim in range(n_dims):
                omega[..., dim] *= 2 * pi / t[dim]

            # compute element-point tensor
            P = V[E] # [n_elem, j+1, d]

            # initialize output dV
            dV = torch.zeros_like(V) # [j+1, n_dims]

            # compute element-point tensor
            P = V[E] # [n_elem, j+1, d]

            # helper functions
            seq = lambda i : permute_seq(i, j+1) # return looped sequences
            def img(x): # tensor times i (assume last dim: real/imag)
                res = x[..., [1, 0]]
                res[..., 0] = -res[..., 0]
                return res

            # loop over element batches
            for idx in range(ceil(n_elem/elem_batch)):
                id_start = idx * elem_batch
                id_end = min((idx+1) * elem_batch, n_elem)
                elem_batch_i = id_end - id_start
                Xi = P[id_start:id_end] # [elem_batch, j+1, d]
                Di = D[id_start:id_end] # [elem_batch, n_channel]
                Ci = C[id_start:id_end] # [elem_batch, 1]
                Ei = E[id_start:id_end] # [elem_batch, j+1]
                CDi = Ci * Di # [elem_batch, n_channel]
                sig = torch.einsum('bjd,...d->bj...', (Xi, omega)) 
                sig = torch.unsqueeze(sig, dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1]
                esig = torch.stack((torch.cos(sig), -torch.sin(sig)), dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 2]
                # specific code for j == 2 and dim == 2 TODO: Add other dimensions
                s01 = 1/(sig - sig[:, seq(1)]).unsqueeze(-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
                s20 = 1/(sig[:, seq(2)] - sig).unsqueeze(-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
                tmp = esig * s01 * s20 # [elem_batch, j+1, dimX, dimY, dimZ, 1, 2]
                S = -torch.sum(tmp, dim=1) # [elem_batch, dimX, dimY, dimZ, 1, 2]
                S[:, 0, 0] = -1 / factorial(j) # HARDCODE dim==2
                tmp = img(tmp) + tmp*(s01 - s20) + tmp[:, seq(1)]*s01 - tmp[:, seq(2)]*s20 # [elem_batch, j+1, dimX, dimY, dimZ, 1, 2]
                tmp.unsqueeze_(-3) # [elem_batch, j+1, dimX, dimY, dimZ, 1(n_dims), 1(n_channel), 2(real/imag)]
                tmp[:, :, 0, 0] = 0
                tmp = tmp * dF.unsqueeze(-3) * omega.unsqueeze(-1).unsqueeze(-1) # [elem_batch, j+1, dimX, dimY, dimZ, n_dims, n_channel, 2]
                tmp = tmp.view(elem_batch_i, j+1, -1, n_dims, n_channel, 2).sum(2) # [elem_batch, j+1, n_dims, n_channel, 2]
                tmp = -(tmp * CDi.unsqueeze(1).unsqueeze(1).unsqueeze(-1)).view(elem_batch_i, j+1, n_dims, -1).sum(-1) # [elem_batch, j+1, n_dims]
                xy = torch.stack((Xi[:, seq(1), 1] - Xi[:, seq(2), 1], - Xi[:, seq(1), 0] + Xi[:, seq(2), 0]), dim=-1) # [elem_batch, j+1, n_dims]
                tmp2 = S * dF # [elem_batch, dimX, dimY, dimZ, n_channel, 2]
                tmp2 = tmp2.sum(-1).view(elem_batch_i, -1, n_channel).sum(1) # [elem_batch, n_channel]
                tmp2 = -((tmp2 * Di).sum(-1).unsqueeze(-1).unsqueeze(-1) * xy) # [elem_batch, j+1, n_dims]
                ddV = coalesce_update(Ei, tmp + tmp2, dV.shape)
                dV += ddV
            if mode == "density":
                dV *= res[0] ** j
        else:
            dV = None
            
        return dV, None, None, None, None, None, None, None

    
class DDSL(nn.Module):
    def __init__(self, res, t, j, elem_batch=100, mode='density'):
        super(DDSL, self).__init__()
        self.res = res
        self.t = t
        self.j = j
        self.elem_batch = elem_batch
        self.mode = mode
    def forward(self, V, E, D):
        return SimplexFT.apply(V,E,D,self.res,self.t,self.j,self.elem_batch,self.mode)
        

In [5]:
r = 256
j = 2
dim = 2
npt = 100
# Vertex matrix
V, E = rand_hull(npt, dim, device="cuda", dtype=torch.float32)

# Density matrix (Random Density 0 to 1)
D = torch.rand(E.shape[0], 1, dtype=V.dtype, device=V.device)
# D = torch.ones(E.shape[0], 1, dtype=V.dtype, device=V.device)

res = [r] * 2
t = [1] * 2

# Sensitivity on each frequency mode
dF = torch.ones(r, int(r/2)+1, 2, dtype=V.dtype) # unit sensitivity

ddsl = DDSL(res, t, j, elem_batch=100, mode='density')

t0 = time()
F = ddsl(V, E, D)
t1 = time()
print("Number of Elements: {}".format(E.shape[0]))
print("Forward Time Lapse: {}".format(t1 - t0))
torch.cuda.empty_cache()

# plot
plot_F(F)

# Backwards pass time
loss = F.view(-1).sum(0)
if V.grad is not None:
    V.grad.zero_()
t0 = time()
loss.backward()
t1 = time()
dV = V.grad
print("Backward Time Lapse: {}".format(t1 - t0))

[!]Warning: zeroing small negative number tensor([ 1.8713e-06,  5.9443e-06,  1.6168e-05,  3.8286e-07,  2.5964e-05,
         4.3660e-06,  0.0000e+00,  2.8821e-06,  5.3140e-06,  2.9441e-05,
         5.2247e-07,  4.5550e-05,  2.6005e-06,  5.5873e-06,  7.4973e-07,
         6.0697e-06, -1.7297e-04,  1.5708e-06,  1.3029e-05,  8.0025e-06,
         6.2355e-07,  2.3916e-06,  1.9990e-05,  7.6866e-05,  1.8350e-05,
         8.2832e-07,  4.3205e-06,  1.3456e-06,  1.8972e-06,  9.1140e-06,
         6.6077e-06,  2.0623e-06,  4.4150e-06,  7.4349e-06,  9.0356e-06,
         1.0471e-04,  2.8621e-06,  3.8440e-05,  1.4201e-05,  5.3398e-06,
         3.3683e-07, -4.9650e-05, -1.4587e-06,  2.7960e-06,  5.0406e-07,
         5.1494e-08,  2.9612e-05,  6.8145e-06,  1.9210e-07,  2.1702e-06,
         4.1765e-06,  4.1937e-07,  4.0861e-07,  1.2375e-07,  2.7657e-06,
         2.1998e-07,  1.8195e-05,  2.5549e-06,  3.9509e-06,  5.7593e-07,
         8.0747e-08,  1.0839e-07,  1.4752e-06,  1.8889e-05,  1.5462e-07,
         

RuntimeError: output with shape [100, 256, 129, 1, 2] doesn't match the broadcast shape [100, 256, 129, 100, 2]

In [ ]:
LU = torch.rand(10,3,3)
det_LU = torch.einsum('...ii->...i', (LU)).prod(-1)

## Setting up

In [ ]:
r = 256
j = 2
# # Vertex matrix
# V_ = np.array([[0.2,0.2],
#                [0.8,0.5],
#                [0.8,0.8],
#                [0.6,0.8]])
# V_ += 1e-4*np.random.rand(*V_.shape)
# V = torch.tensor(V_, dtype=torch.float64, requires_grad=True, device="cuda")
# # Element matrix
# E = torch.LongTensor([[0,1,2],
#                       [2,3,0]])
# # Density matrix (Unit Density)
# D = torch.ones(E.shape[0], 1, dtype=V.dtype)

# Vertex matrix
V, E = rand_hull(200, 2, device="cuda", dtype=torch.float64)

# Density matrix (Random Density 0 to 1)
# D = torch.rand(E.shape[0], 1, dtype=V.dtype, device=V.device)
D = torch.ones(E.shape[0], 1, dtype=V.dtype)

res = [r] * 2
t = [1] * 2

E, D = E.cuda(), D.cuda()

# Sensitivity on each frequency mode
dF = torch.ones(r, int(r/2)+1, 2, dtype=V.dtype) # unit sensitivity

## Test backward

In [ ]:
# Analytical Adjoint solution
ddsl = DDSL(res,t,j,100,'density')
F = ddsl(V,E,D)
loss = F.view(-1).sum(0)
if V.grad is not None:
    V.grad.zero_()
loss.backward()
dV = V.grad
print("Analytical Gradient:")
print(dV)

print("Finite Difference Gradient (New):")
# Finite difference approximation
delta = 1e-6
dV_fd_all = torch.zeros(*(list(V.shape)+list(dF.shape)+[2]), dtype=V.dtype).cuda()
for ii in range(V.shape[0]):
    for jj in range(V.shape[1]):
        V_p = V.clone()
        V_m = V.clone()
        V_p[ii, jj] += delta
        V_m[ii, jj] -= delta
        Freq_p = ddsl(V_p, E, D)
        Freq_m = ddsl(V_m, E, D)
        dV_fd_all[ii,jj] = (Freq_p - Freq_m) / delta / 2
dV_fd = dV_fd_all.view(V.shape[0], V.shape[1], -1).sum(-1)
print(dV_fd)
# print(dV/dV_fd)
print(dV / dV_fd)
plot_F(F)

## Test forward code

In [ ]:
ddsl = DDSL(res, t, j, elem_batch=100, mode='density')

dim = 2
t0 = time()
F = ddsl(V, E, D)
print("Time Lapse: {}".format(time() - t0))
torch.cuda.empty_cache()

# plot
F = F.squeeze()
if torch.sum(torch.isnan(F)) > 0:
    nnan = torch.sum(torch.isnan(F))
    F[torch.isnan(F)] = 0
    print("Padding {} nan terms to zero.".format(nnan))

F_ = F.cpu().detach().numpy()
f_ = torch.irfft(F, dim, signal_sizes=res).squeeze().cpu().detach().numpy()
plt.figure(figsize=(6,6))
plt.imshow(f_.T, origin='lower')

plt.colorbar()
plt.show()

## Test backward code

## Scratch